# End-to-End FINN Flow for a Simple Convolutional Net
-----------------------------------------------------------------

In this notebook, we will go through the FINN steps needed to take a binarized convolutional network all the way down to a heterogeneous streaming dataflow accelerator running on the FPGA. 

It's recommended to go through the simpler [end-to-end notebook for a fully connected network](tfc_end2end_example.ipynb) first, since many steps here are very similar and we will focus on what is done differently for convolutions.

This notebook is quite lengthy, and some of the cells (involving Vivado synthesis) may take up to an hour to finish running. To let you save and resume your progress, we will save the intermediate ONNX models that are generated in the various steps to disk, so that you can jump back directly to where you left off.

## Quick Introduction to the CNV-w1a1 Network

The particular quantized neural network (QNN) we will be targeting in this notebook is referred to as CNV-w1a1 and it classifies 32x32 RGB images into one of ten CIFAR-10 classes. All weights and activations in this network are quantized to bipolar values (either -1 or +1), with the exception of the input (which is RGB with 8 bits per channel) and the final output (which is 32-bit numbers). It first appeared in the original [FINN paper](https://arxiv.org/abs/1612.07119) from ISFPGA'17 with the name CNV, as a variant of the binarized convolutional network from the [BinaryNet paper](https://arxiv.org/abs/1602.02830), in turn inspired by the VGG-11 topology which was the runner-up for the 2014 [ImageNet Large Scale Visual Recognition Challenge](http://www.image-net.org/challenges/LSVRC/).


You'll have a chance to interactively examine the layers that make up the network in Netron in a moment, so that's enough about the network for now. 

## Quick Recap of the End-to-End Flow

The FINN compiler comes with many *transformations* that modify the ONNX representation of the network according to certain patterns. This notebook will demonstrate a *possible* sequence of such transformations to take a particular trained network all the way down to hardware, as shown in the figure below.

![](finn-design-flow-example.svg)

The white fields show the state of the network representation in the respective step. The colored fields represent the transformations that are applied to the network to achieve a certain result. The diagram is divided into 5 sections represented by a different color, each of it includes several flow steps. The flow starts in top left corner with Brevitas export (green section), followed by the preparation of the network (blue section) for the Vivado HLS synthesis and Vivado IPI stitching (orange section), and finally building a PYNQ overlay bitfile and testing it on a PYNQ board (yellow section).
There is an additional section for functional verification (red section) on the left side of the diagram, which we will not cover in this notebook. For details please take a look in the verification notebook which you can find [here](tfc_end2end_verification.ipynb)


We will use the helper function `showInNetron` to show the ONNX model at the current transformation step. The Netron displays are interactive, but they only work when running the notebook actively and not on GitHub (i.e. if you are viewing this on GitHub you'll only see blank squares).

In [1]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
    
build_dir = "/workspace/finn/final/X29"

## 1. Brevitas Export, FINN Import and Tidy-Up


Similar to what we did in the TFC-w1a1 end-to-end notebook, we will start by exporting the [pretrained CNV-w1a1 network](https://github.com/maltanar/brevitas_cnv_lfc) to ONNX, importing that into FINN and running the "tidy-up" transformations to have a first look at the topology.

In [2]:
import onnx
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.double_to_single_float import DoubleToSingleFloat
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames
import enum

import torch
import torch.nn.functional as F
import torch.nn as nn
from brevitas.quant_tensor import pack_quant_tensor
import brevitas.nn as qnn
from brevitas.core.quant import QuantType
from brevitas.core.restrict_val import RestrictValueType
from brevitas.core.scaling import ScalingImplType
from brevitas.core.stats import StatsOp

Infer_shape used


In [3]:
class ZeroAct(nn.Module):
    def __init__(self, quant_type, bit_width, scaling_impl_type=ScalingImplType.CONST,
                 restrict_scaling_type=RestrictValueType.LOG_FP):
        super(ZeroAct, self).__init__()
        self._min_val_act = 0
        self._max_val_act = 255
        self._min_val = 0
        self._max_val = 255
        self._act = qnn.QuantHardTanh(scaling_impl_type=scaling_impl_type, restrict_scaling_type=restrict_scaling_type,
                                      min_val=self._min_val_act, max_val=self._max_val_act, quant_type=quant_type,
                                      bit_width=bit_width)
        self._scale = (self._max_val - self._min_val) / (self._max_val_act - self._min_val_act)

    def forward(self, x):
        x = self._act(x / self._scale - 1)
        x = self._scale * x
        x = x + self._scale
        return x

In [4]:
class NetType(enum.Enum):
    FINN_W2_A2 = 25
    FINN_W2_A2_X2 = 26
    FINN_W2_A2_X3 = 27


class MyQuantReLU(nn.Module):
    def __init__(self, max_val, quant_type, bit_width, scaling_impl_type=ScalingImplType.CONST,
                 restrict_scaling_type=RestrictValueType.LOG_FP):
        super(MyQuantReLU, self).__init__()
        self._min_val_act = -1
        self._max_val_act = 1 - 2 / (2 ** bit_width)
        self._min_val = 0
        self._max_val = max_val
        self._act = qnn.QuantHardTanh(scaling_impl_type=scaling_impl_type, restrict_scaling_type=restrict_scaling_type,
                                      min_val=self._min_val_act, max_val=self._max_val_act, quant_type=quant_type,
                                      bit_width=bit_width)
        self._scale = (self._max_val - self._min_val) / (self._max_val_act - self._min_val_act)

    def forward(self, x):
        x = self._act(x / self._scale - 1)
        x = self._scale * x
        x = x + self._scale
        return x


In [5]:
class SiamNet(nn.Module):
    def __init__(self):
        super(SiamNet, self).__init__()
        chan_mult = 0.25
        
        # FINN_W2_A2_X29
        FIRST_LAYER_QUANT_TYPE = QuantType.INT
        FIRST_LAYER_BIT_WIDTH = 2
        WEIGHT_QUANT_TYPE = QuantType.INT
        WEIGHT_BIT_WIDTH = 2
        LAST_LAYER_QUANT_TYPE = QuantType.INT
        LAST_LAYER_BIT_WIDTH = 2
        ACTIVATION_QUANT_TYPE = QuantType.INT
        ACTIVATION_BIT_WIDTH = 2
        ACTIVATION_SCALING_IMPL_TYPE = ScalingImplType.CONST
        ACTIVATION_MAX_VAL = 6

        self.conv_features = nn.Sequential(
            ZeroAct(quant_type=QuantType.INT,
                bit_width=8,
                scaling_impl_type=ScalingImplType.CONST,
                restrict_scaling_type=RestrictValueType.POWER_OF_TWO),
            qnn.QuantConv2d(in_channels=3, out_channels=int(chan_mult*96), kernel_size=11,
                weight_quant_type=FIRST_LAYER_QUANT_TYPE,
                weight_bit_width=FIRST_LAYER_BIT_WIDTH,
                bias=False),

            nn.BatchNorm2d(int(chan_mult*96), eps=1e-4),
            MyQuantReLU(ACTIVATION_MAX_VAL, ACTIVATION_QUANT_TYPE, ACTIVATION_BIT_WIDTH,
            ACTIVATION_SCALING_IMPL_TYPE),

            nn.MaxPool2d(kernel_size=2, stride=2),

            qnn.QuantConv2d(int(chan_mult*96), int(chan_mult*256), 5, weight_quant_type=WEIGHT_QUANT_TYPE,
                weight_bit_width=WEIGHT_BIT_WIDTH, bias=False),
            nn.BatchNorm2d(int(chan_mult*256), eps=1e-4),
            MyQuantReLU(ACTIVATION_MAX_VAL, ACTIVATION_QUANT_TYPE, ACTIVATION_BIT_WIDTH,
            ACTIVATION_SCALING_IMPL_TYPE),

            qnn.QuantConv2d(int(chan_mult*256), int(chan_mult*384), 3, weight_quant_type=WEIGHT_QUANT_TYPE,
                weight_bit_width=WEIGHT_BIT_WIDTH, bias=False),
            nn.BatchNorm2d(int(chan_mult*384), eps=1e-4),
            MyQuantReLU(ACTIVATION_MAX_VAL, ACTIVATION_QUANT_TYPE, ACTIVATION_BIT_WIDTH,
            ACTIVATION_SCALING_IMPL_TYPE),

            nn.MaxPool2d(kernel_size=3, stride=3),

            qnn.QuantConv2d(int(chan_mult*384), int(chan_mult*384), 3, weight_quant_type=WEIGHT_QUANT_TYPE,
                weight_bit_width=WEIGHT_BIT_WIDTH, bias=False),

            nn.BatchNorm2d(int(chan_mult*384), eps=1e-4),
            MyQuantReLU(ACTIVATION_MAX_VAL, ACTIVATION_QUANT_TYPE, ACTIVATION_BIT_WIDTH,
            ACTIVATION_SCALING_IMPL_TYPE),

            qnn.QuantConv2d(int(chan_mult*384), int(chan_mult*256), 3, weight_quant_type=LAST_LAYER_QUANT_TYPE,
                weight_bit_width=LAST_LAYER_BIT_WIDTH, bias=False)
        )

        # adjust layer as in the original SiamFC in matconvnet
        self.adjust = nn.Conv2d(1, 1, 1, 1)
        #self.adjust = qnn.QuantConv2d(1, 1, 1, 1, weight_quant_type=WEIGHT_QUANT_TYPE, weight_bit_width=WEIGHT_BIT_WIDTH, bias=False)
    
        # initialize weights
        self._initialize_weight()

    def forward(self, x):
        x_feat = self.conv_features(x)
        return x_feat

    def xcorr(self, z, x):
        """
        correlation layer as in the original SiamFC (convolution process in fact)
        """
        batch_size_x, channel_x, w_x, h_x = x.shape
        x = torch.reshape(x, (1, batch_size_x * channel_x, w_x, h_x))

        # group convolution
        out = F.conv2d(x, z, groups=batch_size_x)

        batch_size_out, channel_out, w_out, h_out = out.shape
        xcorr_out = torch.reshape(out, (channel_out, batch_size_out, w_out, h_out))

        return xcorr_out

    def _initialize_weight(self):
        """
        initialize network parameters
        """
        tmp_layer_idx = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                tmp_layer_idx = tmp_layer_idx + 1
                if tmp_layer_idx < 6:
                    # kaiming initialization
                    nn.init.kaiming_normal_(m.weight.data, mode='fan_out', nonlinearity='relu')
                else:
                    # initialization for adjust layer as in the original paper
                    m.weight.data.fill_(1e-3)
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def weight_loss(self, prediction, label, weight):
        """
        weighted cross entropy loss
        """

        # if self.network_type != NetType.BASELINE:
        loss = F.binary_cross_entropy_with_logits(prediction, label, weight, reduction='mean')
        # else:
        #     loss = F.binary_cross_entropy_with_logits(prediction, label, weight,
        #                                               size_average=False) / self.config.batch_size

        return loss


In [6]:
import onnx
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo

#checkpoint_loc_conv = "/workspace/finn/siam_checkpoint/siam_conv.tar"
#conv_shape = (1,3,256,256)
conv_shape = (1, 3, 130, 130)
net = SiamNet()
checkpoint_loc = build_dir + "/SiamFC_30_model.pth"
in_shape = conv_shape
checkpoint = torch.load(checkpoint_loc, map_location="cpu")['state_dict']
net.load_state_dict(checkpoint)
cnv = net.eval()
bo.export_finn_onnx(cnv, in_shape, build_dir+"/siam_cnv.onnx")

model = ModelWrapper(build_dir + "/siam_cnv.onnx")
model = model.transform(DoubleToSingleFloat())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model.save(build_dir + "/siam_cnv_tidy.onnx")

Now that the model is exported, let's have a look at its layer structure with Netron. Remember that the visualization below is interactive, you can click on the individual nodes and view the layer attributes, trained weights and so on.

In [7]:
showInNetron(build_dir+"/siam_cnv_tidy.onnx")

Serving '/workspace/finn/final/X29/siam_cnv_tidy.onnx' at http://0.0.0.0:8081


You can see that the network is composed of a repeating convolution-convolution-maxpool layer pattern to extract features using 3x3 convolution kernels (with weights binarized) and `Sign` activations, followed by fully connected layers acting as the classifier. Also notice the initial `MultiThreshold` layer at the beginning of the network, which is quantizing float inputs to 8-bit ones.

## 2. How FINN Implements Convolutions: Lowering and Streamlining

In FINN, we implement convolutions with the *lowering* approach: we convert them to matrix-matrix multiply operations, where one of the matrices is generated by sliding a window over the input image. You can read more about the sliding window operator and how convolution lowering works [in this notebook](https://github.com/maltanar/qnn-inference-examples/blob/master/3-convolutional-binarized-gtsrb.ipynb). The streaming dataflow architecture we will end up with is going to look something like this figure from the [FINN-R paper](https://arxiv.org/abs/1809.04570):

![](cnv-mp-fc.png)

Note how the convolution layer looks very similar to the fully connected one in terms of the matrix-vector-threshold unit (MVTU), but now the MVTU is preceded by a sliding window unit that produces the matrix from the input image. All of these building blocks, including the `MaxPool` layer you see in this figure, exist as templated Vivado HLS C++ functions in [finn-hlslib](https://github.com/Xilinx/finn-hlslib).


To target this kind of hardware architecture with our network we'll apply a convolution lowering transformation, in addition to streamlining. You may recall the *streamlining transformation* that we applied to the TFC-w1a1 network, which is a series of mathematical simplifications that allow us to get rid of floating point scaling operations by implementing few-bit activations as thresholding operations. **The current implementation of streamlining is highly network-specific and may not work for your network if its topology is very different than the example network here. We hope to rectify this in future releases.**

In [8]:
from finn.transformation.streamline import Streamline
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveAddMulPastIm2Col

model = ModelWrapper(build_dir + "/siam_cnv_tidy.onnx")
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(MoveAddMulPastIm2Col())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(ConvertBipolarMatMulToXnorPopcount())
model = model.transform(Streamline())
model.save(build_dir + "/siam_cnv_streamlined.onnx")


In [9]:
showInNetron(build_dir+"/siam_cnv_streamlined.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/final/X29/siam_cnv_streamlined.onnx' at http://0.0.0.0:8081


We won't go into too much detail about what happens in each transformation and why they are called in the particular order they are (feel free to visualize the intermediate steps using Netron yourself if you are curious) but here is a brief summmmary:

* `Streamline` moves floating point scaling and addition operations closer to the input of the nearest thresholding activation and absorbs them into thresholds
* `LowerConvsToMatMul` converts ONNX `Conv` nodes into sequences of `Im2Col, MatMul` nodes as discussed above. `Im2Col` is a custom FINN ONNX high-level node type that implements the sliding window operator.
* `MakeMaxPoolNHWC` and `AbsorbTransposeIntoMultiThreshold` convert the *data layout* of the network into the NHWC data layout that finn-hlslib primitives use. NCHW means the tensor dimensions are ordered as `(N : batch, H : height, W : width, C : channels)` (assuming 2D images). The ONNX standard ops normally use the NCHW layout, but the ONNX intermediate representation itself does not dictate any data layout.
* You may recall `ConvertBipolarMatMulToXnorPopcount` from the TFC-w1a1 example, which is needed to implement bipolar-by-bipolar (w1a1) networks correctly using finn-hlslib.

Let's visualize the streamlined and lowered network with Netron. Observe how all the `Conv` nodes have turned into pairs of `Im2Col, MatMul` nodes, and many nodes including `BatchNorm, Mul, Add` nodes have disappeared and replaced with `MultiThreshold` nodes.

In [10]:
showInNetron(build_dir+"/siam_cnv_streamlined.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/final/X29/siam_cnv_streamlined.onnx' at http://0.0.0.0:8081


## 3. Partitioning, Conversion to HLS Layers and Folding

The next steps will be (again) very similar to what we did for the TFC-w1a1 network. We'll first convert the layers that we can put into the FPGA into their HLS equivalents and separate them out into a *dataflow partition*:


In [11]:
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.custom_op.registry import getCustomOp

# choose the memory mode for the MVTU units, decoupled or const
#mem_mode = "decoupled"
#mem_mode = "MVTU"
mem_mode = "const"

model = ModelWrapper(build_dir + "/siam_cnv_streamlined.onnx")
model = model.transform(to_hls.InferBinaryStreamingFCLayer(mem_mode))
model = model.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode))
model = model.transform(to_hls.InferConvInpGen())
model = model.transform(to_hls.InferStreamingMaxPool())
# get rid of Reshape(-1, 1) operation between hlslib nodes
model.save(build_dir + "/tmp.onnx")
model = model.transform(RemoveCNVtoFCFlatten())
parent_model = model.transform(CreateDataflowPartition())
parent_model.save(build_dir + "/siam_cnv_dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
# save the dataflow partition with a different name for easier access
dataflow_model = ModelWrapper(dataflow_model_filename)
dataflow_model.save(build_dir + "/siam_cnv_dataflow_model.onnx")

Notice the additional `MoveReshape` transformation that was not used for TFC-w1a1. In the last Netron visualization you may have noticed a `Reshape` operation towards the end of the network where the convolutional part of the network ends and the fully-connected layers started. That `Reshape` is essentialy a tensor flattening operation, which we can remove for the purposes of hardware implementation. We can examine the contents of the dataflow partition with Netron, and observe the `ConvolutionInputGenerator`, `StreamingFCLayer_Batch` and `StreamingMaxPool_Batch` nodes that implement the sliding window, matrix multiply and maxpool operations in hlslib. *Note that the StreamingFCLayer instances following the ConvolutionInputGenerator nodes are really implementing the convolutions, despite the name. The final three StreamingFCLayer instances implement actual FC layers.*

In [12]:
showInNetron(build_dir + "/tmp.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/final/X29/tmp.onnx' at http://0.0.0.0:8081


In [13]:
showInNetron(build_dir + "/siam_cnv_dataflow_parent.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/final/X29/siam_cnv_dataflow_parent.onnx' at http://0.0.0.0:8081


In [14]:
showInNetron(build_dir + "/siam_cnv_dataflow_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/final/X29/siam_cnv_dataflow_model.onnx' at http://0.0.0.0:8081


Now we have to set the *folding factors* for certain layers to adjust the performance of our accelerator, similar to the TFC-w1a1 example. We'll also set the desired FIFO depths around those layers, which are important to achieve full throughput in the accelerator.

In [15]:
from finn.transformation.fpgadataflow.insert_dwc import InsertDWC
from finn.transformation.fpgadataflow.insert_tlastmarker import InsertTLastMarker
from finn.transformation.fpgadataflow.insert_fifo import InsertFIFO

model = ModelWrapper(build_dir + "/siam_cnv_dataflow_model.onnx")
fc_layers = model.get_nodes_by_op_type("StreamingFCLayer_Batch")

# each tuple is (PE, SIMD, in_fifo_depth) for a layer
# # there are 7 StreamingFCLayer_Batch
# folding = [
#     (16, 1, 32),
#     (16, 32, 32),
#     (16, 32, 32),
#     (16, 32, 32),
#     (4, 32, 81),
#     (1, 32, 2),
#     (1, 4, 2),
#     (1, 4, 2),
# ]
folding = [
    (4, 1, 32),
    (4, 4, 32),
    (4, 4, 32),
    (4, 4, 32),
    (4, 4, 81),
    (1, 4, 2),
    (1, 4, 2),
    (1, 4, 2),
]

for fcl, (pe, simd, ififodepth) in zip(fc_layers, folding):
    fcl_inst = getCustomOp(fcl)
    fcl_inst.set_nodeattr("PE", pe)
    fcl_inst.set_nodeattr("SIMD", simd)
    fcl_inst.set_nodeattr("inFIFODepth", ififodepth)

# use same SIMD values for the sliding window operators
swg_layers = model.get_nodes_by_op_type("ConvolutionInputGenerator")
for i in range(len(swg_layers)):
    print("i = {}".format(i))
    swg_inst = getCustomOp(swg_layers[i])
    simd = folding[i][1]
    swg_inst.set_nodeattr("SIMD", simd)

model = model.transform(InsertDWC())
model = model.transform(InsertFIFO())
model = model.transform(InsertTLastMarker())
model = model.transform(GiveUniqueNodeNames())
model.save(build_dir + "/siam_cnv_folded.onnx")

i = 0
i = 1
i = 2
i = 3
i = 4


Below we visualize in Netron to observe the `StreamingDataWidthConverter` and `StreamingFIFO` nodes that have been inserted into graph, as well as the folding factors in the `PE` and `SIMD` attributes of each `StreamingFCLayer_Batch`.

In [16]:
showInNetron(build_dir + "/siam_cnv_folded.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/final/X29/siam_cnv_folded.onnx' at http://0.0.0.0:8081


Our network is now ready and we can start with the hardware generation.

## 4. Hardware Generation

From this point onward, the steps we have to follow do not depend on the particular network and will be exactly the same as the TFC-w1a1 example. We first proceed with HLS synthesis, **which may take 10-20 minutes depending on your host computer**.

In [17]:
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.util.basic import pynq_part_map

test_pynq_board = "ZCU104"
test_fpga_part = pynq_part_map[test_pynq_board]
target_clk_ns = 5

model = ModelWrapper(build_dir + "/siam_cnv_folded.onnx")

In [18]:
model = model.transform(PrepareIP(test_fpga_part, target_clk_ns))

In [19]:
model = model.transform(HLSSynthIP(0))

In [20]:
model.save(build_dir + "/siam_cnv_ipgen.onnx")

Once the HLS synthesis is complete, we can stitch together the generated IP blocks into a larger IP that is the implementation of our network:

In [21]:
from finn.transformation.fpgadataflow.replace_verilog_relpaths import (
    ReplaceVerilogRelPaths,
)
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP

model = ModelWrapper(build_dir + "/siam_cnv_ipgen.onnx")
model = model.transform(ReplaceVerilogRelPaths())
model = model.transform(CreateStitchedIP(test_fpga_part))
model.save(build_dir + "/siam_cnv_ipstitch.onnx")

Finally, we create a PYNQ project that includes the hardware "shell" that will support our accelerator, including the data movers, and run Vivado synthesis, **which may take around 30 minutes depending on your host computer.**

*If you'd like to watch the progress, you can open the generated project file (printed below) with the Vivado GUI.*

In [22]:
from finn.transformation.fpgadataflow.make_pynq_proj import MakePYNQProject
from finn.transformation.fpgadataflow.synth_pynq_proj import SynthPYNQProject

model = ModelWrapper(build_dir + "/siam_cnv_ipstitch.onnx")
model = model.transform(MakePYNQProject(test_pynq_board))
vivado_proj = model.get_metadata_prop("vivado_pynq_proj")
print("Vivado synthesis project is at %s/resizer.xpr" % vivado_proj)
model.save(build_dir + "/siam_cnv_pynqproj.onnx")

Vivado synthesis project is at /tmp/finn_dev_vision/vivado_pynq_proj_ap9otkm1/resizer.xpr


In [23]:
model = ModelWrapper(build_dir + "/siam_cnv_pynqproj.onnx")
model = model.transform(SynthPYNQProject())
model.save(build_dir + "/siam_cnv_synth.onnx")

## 5. Deployment and Remote Execution

Now that we're done with the hardware generation, we can generate a Python driver for accelerator and copy the necessary files onto our PYNQ board.

In [24]:
import os
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver
from finn.transformation.fpgadataflow.make_deployment import DeployToPYNQ

# set up the following values according to your own environment
# FINN will use ssh to deploy and run the generated accelerator
ip = os.getenv("PYNQ_IP", "192.168.1.99")
username = os.getenv("PYNQ_USERNAME", "xilinx")
password = os.getenv("PYNQ_PASSWORD", "xilinx")
port = os.getenv("PYNQ_PORT", 22)
target_dir = os.getenv("PYNQ_TARGET_DIR", "/home/xilinx/finn")

model = ModelWrapper(build_dir + "/siam_cnv_synth.onnx")
model = model.transform(MakePYNQDriver())
model = model.transform(DeployToPYNQ(ip, port, username, password, target_dir))
model.save(build_dir + "/siam_cnv_pynq_deploy.onnx")

In [25]:
! sshpass -p {password} ssh {username}@{ip} -p {port} 'ls -l {target_dir}/*'

We only have two more steps to be able to remotely execute the deployed bitfile with some test data from the CIFAR-10 dataset. Let's load up some test data that comes bundled with FINN -- and before you ask, that's supposed to be a cat (CIFAR-10 class number 3).

In [26]:
import pkg_resources as pk
import matplotlib.pyplot as plt
import numpy as np

fn = pk.resource_filename("finn", "data/cifar10/cifar10-test-data-class3.npz")
x = np.load(fn)["arr_0"].astype(np.float32)
x = x / 255
plt.imshow(x.reshape(3, 32,32).transpose(1, 2, 0))
plt.show()

<Figure size 640x480 with 1 Axes>

Recall that we partitioned our original network into a parent graph that contained the non-synthesizable nodes and a child graph that contained the bulk of the network, which we turned into a bitfile. We'll load up the parent graph, modify the `StreamingDataflowPartition` node so that it points to the deployed ONNX graph.

In [27]:
# point to the PYNQ-deployed model as the StreamingDataflowPartition in the parent
parent_model = ModelWrapper(build_dir+"/siam_cnv_dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
sdp_node.set_nodeattr("model", build_dir + "/siam_cnv_pynq_deploy.onnx")
parent_model.save(build_dir+"/siam_cnv_dataflow_parent_with_remote_bitfile_exec.onnx")

Finally, we can call `execute_onnx` on the parent graph, which will internally call remote execution with the bitfile once the `StreamingDataflowPartition` node is reached, grab the results, then continue executing the last portion of the network. 

In [28]:
import numpy as np
from finn.core.onnx_exec import execute_onnx
iname = parent_model.graph.input[0].name
oname = parent_model.graph.output[0].name
ishape = parent_model.get_tensor_shape(iname)
input_dict = {iname: x.reshape(ishape)}
ret = execute_onnx(parent_model, input_dict, True)

ValueError: cannot reshape array of size 3072 into shape (1,3,130,130)

We'll pass the output of the network through a softmax function to interpret it as probabilities, and plot the per-class probabilities as a bar chart.

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

logits = ret[oname].flatten()
prob = softmax(logits)

classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

plt.figure(figsize=(20, 3)) 
plt.bar(classes, prob)

We see that the network correctly predicts this as a class 3 ("cat") with high probability. This concludes our tutorial on how to take a convolutional BNN all the way down to hardware with FINN, and execute it remotely on a PYNQ board.